## OSRM (Open Source Routing Machine)

**Author**: SADDIK Imad
<br/>
**Date**: 26/12/2024

---

**Table of contents**<a id='toc0_'></a>    
- [Route service](#toc1_)    
  - [Setup](#toc1_1_)    
  - [Using the service](#toc1_2_)    
    - [General options](#toc1_2_1_)    
    - [Making the request](#toc1_2_2_)    
    - [Plotting the path](#toc1_2_3_)    
    - [Exploring legs](#toc1_2_4_)    
    - [Exploring the intersections](#toc1_2_5_)    
    - [Exploring the cost of the route](#toc1_2_6_)    
    - [Alternative routes](#toc1_2_7_)    

---

# <a id='toc1_'></a>[Route service](#toc0_)

Use this service if you want to find the fastest route between coordinates. Read more about it [here](https://project-osrm.org/docs/v5.24.0/api/?language=cURL#route-service).

## <a id='toc1_1_'></a>[Setup](#toc0_)

I selected four points from the map, designating one as the center. Using the `Folium` package, I will visualize these points on the map.

In [11]:
import folium
from folium import Map
from pydantic import BaseModel

class Point(BaseModel):
    latitude: float
    longitude: float

def get_folium_map(center_point: Point, points: list[Point], zoom_level: int = 12) -> Map:
    folium_map = folium.Map(
        location=[center_point.latitude, center_point.longitude], zoom_start=zoom_level)

    for point in points:
        folium.Marker(location=[point.latitude, point.longitude],
                      popup='Point').add_to(folium_map)

    return folium_map

point_1 = Point(latitude=22.0064, longitude=113.3763)
point_2 = Point(latitude=22.1190, longitude=113.3572)
point_3 = Point(latitude=22.0653, longitude=113.1167)
center_point = Point(latitude=22.0636, longitude=113.2834)

folium_map = get_folium_map(center_point, [point_1, point_2, point_3])
folium_map

In [3]:
#用户自定义点位
import folium
from folium import Map
from folium.plugins import Draw
from pydantic import BaseModel

class Point(BaseModel):
    latitude: float
    longitude: float

def get_folium_map(center_point: Point, zoom_level: int = 14) -> Map:
    folium_map = folium.Map(
        location=[center_point.latitude, center_point.longitude], zoom_start=zoom_level)
    
    # 添加 Draw 插件，支持用户点击添加标记点
    draw = Draw(
        position='topleft',  # 工具栏位置
        draw_options={
            'marker': {'shapeOptions': {'color': '#ff0000'}},  # 标记样式自定义
        },
        edit_options={
            'remove': True,  # 允许删除标记
        }
    ).add_to(folium_map)
    
    return folium_map

# 示例中心点（可以根据需要调整）
center_point = Point(latitude=22.0636, longitude=113.2834)

folium_map = get_folium_map(center_point)
folium_map

## <a id='toc1_2_'></a>[Using the service](#toc0_)

### <a id='toc1_2_1_'></a>[General options](#toc0_)

The general options are used in every OSRM service, read more about them [here](https://project-osrm.org/docs/v5.5.1/api/?language=cURL#general-options). Here is a table that summarizes the options with their description.


| Parameter | Description |
|-----------|-------------|
| service | One of the following values: `route`, `nearest`, `table`, `match`, `trip`, `tile` |
| version | Version of the protocol implemented by the service. `v1` for all OSRM 5.x installations |
| profile | Mode of transportation, is determined statically by the Lua profile that is used to prepare the data using `osrm-extract`. Typically `car`, `bike` or `foot` if using one of the supplied profiles. |
| coordinates | String of format `{longitude},{latitude};{longitude},{latitude};{longitude},{latitude} ...]` or `polyline({polyline})`. |
| format | Only `json` is supported at the moment. This parameter is optional and defaults to `json`. |

### <a id='toc1_2_2_'></a>[Making the request](#toc0_)

In [4]:
service = 'route'
version = 'v1'
# profile = 'driving'
profile = 'cycling'
# profile = 'walking'
host = 'http://localhost:5000'

The route endpoint follows this format: `/route/v1/{profile}/{coordinates}?alternatives={true|false}&steps={true|false}&geometries={polyline|polyline6|geojson}&overview={full|simplified|false}&annotations={true|false}`.

To request the shortest path between three points, the order of the points is crucial. If you want the route from point 1 to point 2 to point 3, make sure to list them in that sequence.

In [5]:
import requests
#注意点的顺序
points = [ point_2, point_1,point_3]
coordinates = ';'.join(
    [f'{point.longitude},{point.latitude}' for point in points])

url = f"{host}/{service}/{version}/{profile}/{coordinates}?alternatives=true&overview=full&steps=true&geometries=geojson"

response = requests.get(url)
response.status_code

200

The `200` status code signifies that the request was successful. Now, let's fetch the route and display it on the map.

In [6]:
data = response.json()
data.keys()

dict_keys(['code', 'routes', 'waypoints'])

In [7]:
routes = data['routes']
print(f"Found {len(routes)} route(s)")

Found 1 route(s)


Since we didn't include the `alternatives` option in the request, the response contains only one route. From this route, we can extract the `coordinates`, the `distance` in meters, and the `duration` in seconds.

In [8]:
#返回routes结果的坐标点、距离、时长
route = routes[0]
route_coordinates = route['geometry']["coordinates"]
route_distance = route["distance"]  # unit: meters
route_duration = route["duration"]  # unit: seconds

route_coordinates[:8]

[[113.357177, 22.119086],
 [113.356749, 22.118987],
 [113.356649, 22.118964],
 [113.356045, 22.11883],
 [113.355842, 22.118784],
 [113.355754, 22.118765],
 [113.355747, 22.118779],
 [113.355737, 22.118792]]

### <a id='toc1_2_3_'></a>[Plotting the path](#toc0_)

The route returned by `OSRM` is in the format `[[longitude, latitude], [longitude, latitude], ...]`. To plot the path on the Folium map, we need to swap the latitude and longitude for each point.

In [9]:
route_coordinates = [[point[1], point[0]] for point in route_coordinates]
route_coordinates[:5]

[[22.119086, 113.357177],
 [22.118987, 113.356749],
 [22.118964, 113.356649],
 [22.11883, 113.356045],
 [22.118784, 113.355842]]

In [12]:
folium_map = get_folium_map(center_point, [point_1, point_2, point_3])
popup_text = f"Distance: {route_distance} meters<br>Duration: {route_duration} seconds"
popup = folium.Popup(popup_text, max_width=300)

folium.PolyLine(
    locations=route_coordinates, color='blue', weight=5, popup=popup).add_to(folium_map)

folium_map

### <a id='toc1_2_4_'></a>[Exploring legs](#toc0_)

Each path is referred to as a `leg`. In our case, there are two legs: one that takes us from point 1 to point 2, and the other that connects point 2 to point 3. Each leg contains multiple steps, where each step provides details about how the route changes at a specific point, such as a turn, a continuation in a straight line, or the start/end of the route.

leg包含steps

Let's draw the 2 legs.

In [ ]:
folium_map = get_folium_map(center_point, [point_1, point_2, point_3])

colors = ['black', 'blue']
#循环 用颜色区分legs
legs = data["routes"][0]["legs"]
for i, leg in enumerate(legs):
    steps = leg["steps"]
    for step in steps:
        geometry = step["geometry"]
        coordinates = [[point[1], point[0]]
                       for point in geometry["coordinates"]]
        color = colors[i % len(colors)]
        popup_text = f"""
        Leg number {i + 1}<br>
        Leg distance: {leg['distance']} meters<br>
        Leg duration: {leg['duration']} seconds<br>
        """
        popup = folium.Popup(popup_text, max_width=300)
        polyline = folium.PolyLine(
            locations=coordinates, color=color, weight=6, popup=popup)
        folium_map.add_child(polyline)

folium_map

Let's focus on a single leg and break it down into its individual steps. Each step provides instructions on where to go, what action to take, and the distance to travel before transitioning to the next step.

To illustrate, let's consider the first two segments of the leg.

In [21]:
folium_map = get_folium_map(
    point_1, [point_1, point_2, point_3], zoom_level=16)

colors = ['black', 'blue']
#legs里的每个steps中所含属性 duration, driving side(行驶侧), instruction（机动指令）,type of manuever(描述该核心动作) ，intersections（交通灯的数量）
legs = data["routes"][0]["legs"]
for leg in legs:
    steps = leg["steps"]
    for i, step in enumerate(steps[:3]):
        geometry = step["geometry"]
        coordinates = [[point[1], point[0]]
                       for point in geometry["coordinates"]]
        color = colors[i % len(colors)]
        popup_text = f"""
        Distance: {step['distance']} meters
        <br>Duration: {step['duration']} seconds
        <br>Name: {step['name']}
        <br>Driving side: {step['driving_side']}
        <br>Instruction: {step['maneuver']['modifier']}
        <br>Type of maneuver: {step['maneuver']['type']}
        <br>Intersections: {len(step['intersections'])}
        """
        popup = folium.Popup(popup_text, max_width=300)
        polyline = folium.PolyLine(
            locations=coordinates, color=color, weight=6, popup=popup)
        folium_map.add_child(polyline)
    break

folium_map

Each segment contains details such as the `distance`, `duration`, `name` of the road, and other relevant information. This helps to describe the specifics of the route for each step, guiding us on the actions to take and the duration and distance of each segment.

In [22]:
from pprint import pprint

leg = data["routes"][0]["legs"][0]
step = leg["steps"][0]
pprint(step)
#- location: [经度, 纬度]，交差点位置。
#- bearings: 数组，路口可达方向的轴承（度数，0°=北，顺时针）。第一个交差点只有 256°（西南方向）。第二个有多个（45° 到 345°，几乎全方向）。
#- entry: 布尔数组，表示从哪些轴承可进入（True=可进入）。第二个交差点有 5 个入口可用。
#- in / out: 进入/离开的轴承索引。这里第一个交差点是起点（out=0），第二个是终点（in=1, out=4，表示从轴承1进入，从轴承4离开）。总共 2 个交差点，表明路段简单

{'distance': 151.1,
 'driving_side': 'right',
 'duration': 137,
 'geometry': {'coordinates': [[113.357177, 22.119086],
                              [113.356749, 22.118987],
                              [113.356649, 22.118964],
                              [113.356045, 22.11883],
                              [113.355842, 22.118784],
                              [113.355754, 22.118765]],
              'type': 'LineString'},
 'intersections': [{'bearings': [256],
                    'entry': [True],
                    'location': [113.357177, 22.119086],
                    'out': 0},
                   {'bearings': [45, 75, 105, 165, 255, 345],
                    'entry': [True, False, True, True, True, True],
                    'in': 1,
                    'location': [113.355842, 22.118784],
                    'out': 4}],
 'maneuver': {'bearing_after': 256,
              'bearing_before': 0,
              'location': [113.357177, 22.119086],
              'modifier': 'left',
 

Let's visualize one step of the route by drawing it on the map. We will display the relevant information as a popup, which can include details such as the distance, duration, road name, etc. Additionally, for the `bearing` angle, we will draw a line in the direction of movement to represent the heading at that point.

In [23]:
import math


def add_compass(folium_map: Map, location: tuple, bearing: int, size: int = 50):

    lat, lon = location
    end_lat = lat + (math.cos(math.radians(bearing)) * size * 1e-5)
    end_lon = lon + (math.sin(math.radians(bearing)) * size * 1e-5)

    folium.Marker(
        location=location,
        icon=folium.DivIcon(
            html="""<div style="font-size: 12px; color: red;">📍</div>"""),
        tooltip=f"Bearing: {bearing}°",
    ).add_to(folium_map)

    folium.PolyLine(
        locations=[location, (end_lat, end_lon)],
        color="red",
        weight=2,
    ).add_to(folium_map)


folium_map = get_folium_map(
    point_1, [point_1, point_2, point_3], zoom_level=16)

colors = ['black', 'blue']
legs = data["routes"][0]["legs"]
for leg in legs[:1]:
    steps = leg["steps"]
    for i, step in enumerate(steps[:1]):
        geometry = step["geometry"]
        coordinates = [[point[1], point[0]]
                       for point in geometry["coordinates"]]
        color = colors[i % len(colors)]

        popup_text = f"""
        Distance: {step['distance']} meters
        <br>Duration: {step['duration']} seconds
        <br>Name: {step['name']}
        <br>Driving side: {step['driving_side']}
        <br>Instruction: {step['maneuver']['modifier']}
        <br>Type of maneuver: {step['maneuver']['type']}
        <br>Intersections: {len(step['intersections'])}
        """
        popup = folium.Popup(popup_text, max_width=300)

        polyline = folium.PolyLine(
            locations=coordinates, color=color, weight=6, popup=popup
        )
        folium_map.add_child(polyline)

        maneuver_location = step["maneuver"]["location"]
        bearing = step["maneuver"].get("bearing_after", 0)
        add_compass(
            folium_map=folium_map,
            location=(maneuver_location[1], maneuver_location[0]),
            bearing=bearing
        )

folium_map

### <a id='toc1_2_5_'></a>[Exploring the intersections](#toc0_)

Let’s explain what the values for `intersection` represent. Starting with the first step in the first leg, we encounter 2 objects, each providing detailed information about the road layout at the intersection.

- `out`: This indicates the index of the outgoing bearing from the intersection. In the first object, `out` is 0 because there is only one available bearing angle. In the second object, `out` is 2, as there are three possible options to choose from.
- `in`: The index of the incoming bearing, representing the road you arrived on.
- `entry`: A list of booleans, where each value corresponds to a `bearing` in the bearings array. A `True` value means the road is accessible, while a `False` value means it is not.
- `bearings`: An array of compass bearings (in degrees) that represent all the possible directions (roads) at this intersection. Each bearing corresponds to a road radiating out from the intersection.
- `location`: The precise [longitude, latitude] coordinates of the intersection.

In [25]:
from pprint import pprint

leg = data["routes"][0]["legs"][0]
step = leg["steps"][0]
intersections = step["intersections"]
pprint(intersections)

[{'bearings': [256],
  'entry': [True],
  'location': [113.357177, 22.119086],
  'out': 0},
 {'bearings': [45, 75, 105, 165, 255, 345],
  'entry': [True, False, True, True, True, True],
  'in': 1,
  'location': [113.355842, 22.118784],
  'out': 4}]


Let's plot the `bearing` directions on the map to visualize what these values represent. The first intersection object contains only one bearing value, indicating that there is only one option to choose from.

In [27]:
import math


def add_compass(
    folium_map: Map,
    location: tuple,
    bearing: int,
    size: int = 50,
    color: str = "red",
    weight: int = 2
):
    lat, lon = location
    end_lat = lat + (math.cos(math.radians(bearing)) * size * 1e-5)
    end_lon = lon + (math.sin(math.radians(bearing)) * size * 1e-5)

    folium.Marker(
        location=location,
        icon=folium.DivIcon(
            html="""<div style="font-size: 12px; color: red;">📍</div>"""),
    ).add_to(folium_map)

    folium.PolyLine(
        locations=[location, (end_lat, end_lon)],
        color=color,
        weight=weight,
        tooltip=f"Bearing: {bearing}°",
    ).add_to(folium_map)


folium_map = get_folium_map(
    point_1, [point_1, point_2, point_3], zoom_level=18)

colors = ['black', 'blue']
legs = data["routes"][0]["legs"]
for leg in legs[:1]:
    steps = leg["steps"]
    for i, step in enumerate(steps[:1]):
        geometry = step["geometry"]
        coordinates = [[point[1], point[0]]
                       for point in geometry["coordinates"]]
        color = colors[i % len(colors)]

        popup_text = f"""
        Distance: {step['distance']} meters
        <br>Duration: {step['duration']} seconds
        <br>Name: {step['name']}
        <br>Driving side: {step['driving_side']}
        <br>Instruction: {step['maneuver']['modifier']}
        <br>Type of maneuver: {step['maneuver']['type']}
        <br>Intersections: {len(step['intersections'])}
        """
        popup = folium.Popup(popup_text, max_width=300)

        polyline = folium.PolyLine(
            locations=coordinates, color=color, weight=6, popup=popup
        )
        folium_map.add_child(polyline)

        intersections = step["intersections"]
        for intersection in intersections[:1]:
            location = intersection["location"]
            bearings = intersection["bearings"]
            for j, bearing in enumerate(bearings):
                allowed_for_traversal = intersection["entry"][j]
                add_compass(
                    folium_map=folium_map,
                    location=(location[1], location[0]),
                    bearing=bearing,
                    color="green" if allowed_for_traversal else "red",
                    weight=5
                )

folium_map

After a few steps, we encounter an intersection with three bearing values. These correspond to going straight, turning back, or turning left. The `entry` value for the first bearing, which corresponds to going back, is set to `False`, meaning we cannot reverse our direction. The other two routes, however, can be taken because their `entry` values are set to `True`.

In [1]:
import math


def add_compass(
    folium_map: Map,
    location: tuple,
    bearing: int,
    size: int = 50,
    color: str = "red",
    weight: int = 2
):
    lat, lon = location
    end_lat = lat + (math.cos(math.radians(bearing)) * size * 1e-5)
    end_lon = lon + (math.sin(math.radians(bearing)) * size * 1e-5)

    folium.Marker(
        location=location,
        icon=folium.DivIcon(
            html="""<div style="font-size: 12px; color: black;">📍</div>"""),
    ).add_to(folium_map)

    folium.PolyLine(
        locations=[location, (end_lat, end_lon)],
        color=color,
        weight=weight,
        tooltip=f"Bearing: {bearing}°"
    ).add_to(folium_map)


folium_map = get_folium_map(
    point_1, [point_1, point_2, point_3], zoom_level=20)

colors = ['black', 'blue']
legs = data["routes"][0]["legs"]
for leg in legs[:1]:
    steps = leg["steps"]
    for i, step in enumerate(steps[:1]):
        geometry = step["geometry"]
        coordinates = [[point[1], point[0]]
                       for point in geometry["coordinates"]]
        color = colors[i % len(colors)]

        popup_text = f"""
        Distance: {step['distance']} meters
        <br>Duration: {step['duration']} seconds
        <br>Name: {step['name']}
        <br>Driving side: {step['driving_side']}
        <br>Instruction: {step['maneuver']['modifier']}
        <br>Type of maneuver: {step['maneuver']['type']}
        <br>Intersections: {len(step['intersections'])}
        """
        popup = folium.Popup(popup_text, max_width=300)

        polyline = folium.PolyLine(
            locations=coordinates, color=color, weight=6, popup=popup
        )
        folium_map.add_child(polyline)

        intersections = step["intersections"]
        for intersection in intersections[1:]:
            location = intersection["location"]
            bearings = intersection["bearings"]
            for j, bearing in enumerate(bearings):
                allowed_for_traversal = intersection["entry"][j]
                add_compass(
                    folium_map=folium_map,
                    location=(location[1], location[0]),
                    bearing=bearing,
                    color="green" if allowed_for_traversal else "red",
                    weight=5
                )

folium_map

NameError: name 'Map' is not defined

### <a id='toc1_2_6_'></a>[Exploring the cost of the route](#toc0_)

Now, let's discuss `weights`. The `weight` value represents the cost of the route based on the profile's weighting algorithm. In this case, the weight is 499.8. With the default profile, the weight corresponds to the route's duration, but this can be modified in the `car` profile, which you can find [here](https://github.com/Project-OSRM/osrm-backend/blob/master/profiles/car.lua#L23).

If you're using a different profile, such as the [bike](https://github.com/Project-OSRM/osrm-backend/blob/master/profiles/bicycle.lua#L22) or [foot](https://github.com/Project-OSRM/osrm-backend/blob/master/profiles/foot.lua#L14) profiles, make sure to adjust the respective files.

In [ ]:
weight = route["weight"]
weight_name = route["weight_name"]
duration = route["duration"]
distance = route["distance"]

print(f"Weight: {weight} - {weight_name}")
print(f"Duration: {duration} seconds")
print(f"Distance: {distance} meters")

Weight: 499.8 - routability
Duration: 499.8 seconds
Distance: 6042.4 meters


### <a id='toc1_2_7_'></a>[Alternative routes](#toc0_)

By default, OSRM provides a single route corresponding to the shortest path. To view alternative routes, you need to enable the `alternatives` option and specify the number of alternatives you'd like to receive.

Note: Although you can request alternative routes, there is no guarantee that they will be available.

In [24]:
import requests

points = [point_1, point_2, point_3]
coordinates = ';'.join(
    [f'{point.longitude},{point.latitude}' for point in points])

url = f"{host}/{service}/{version}/{profile}/{
    coordinates}?alternatives=3&overview=full&steps=true&geometries=geojson"

response = requests.get(url)
data = response.json()
routes = data['routes']
print(f"Found {len(routes)} route(s)")

SyntaxError: EOL while scanning string literal (904761405.py, line 7)

The note confirms that even with `alternatives` set to 3, only one route was returned. However, if you're interested, you can modify certain values in the source code to force OSRM to return an alternative route. For more details, check this [thread](https://github.com/Project-OSRM/osrm-backend/issues/5663).